# Creating SQL Databases

### See Your Tables

* https://www.mysql.com/products/workbench/
* https://dbeaver.io/


### Schemas

https://dbdiagram.io/home

Creating SQL DBs in a few ways!! 

1. Pandas' `.to_sql` method
2. Manually via cursor and `INSERT INTO`
3. On an AWS RDB

In [ ]:
import pandas as pd

In [ ]:
# importing our sample data

customers = pd.read_csv('data/customers.csv')
order_details = pd.read_csv('data/orderdetails.csv')
orders = pd.read_csv('data/orders.csv')
products = pd.read_csv('data/products.csv')

display(customers.head(), order_details.head())

## Using Pandas' `.to_sql()` method

In [ ]:
from sqlalchemy import create_engine

# must start with 'sqlite:///' for a relative path
engine = create_engine('sqlite:///sales.db', echo=True) 
# echo determines whether actions are output

In [ ]:
# as a function
def create_sql_table(df, table_name, engine):
    df.to_sql(table_name, con=engine, if_exists='replace')

In [ ]:
create_sql_table(customers, 'customers', engine)

In [ ]:
create_sql_table(order_details, 'order_details', engine)

### Re-reading in our data

In [ ]:
result = engine.execute('SELECT * FROM customers;')
pd.DataFrame(result.fetchall(), columns = result.keys())

## AWS RDS (Relational Database System)

Note: this database just lives on AWS, you can do this same process on a local database!

Pros and cons of these vs `.to_sql()`:
* You can configure your DB more professionally
* Fairly manual process

Setting up a database instance: https://aws.amazon.com/getting-started/hands-on/create-mysql-db/ 

**It is extremely important that you read all the instructions in the linked page so everything is set up correctly!!!**

In [ ]:
# !pip install mysql-connector-python

In [ ]:
import mysql.connector
from mysql.connector import errorcode

In [ ]:
# connecting to your database
cnx = mysql.connector.connect(
    host = 'test-db1.cpizore09tpz.us-east-2.rds.amazonaws.com',
    user = 'admin',
    passwd = 'testing123'
)

# cursor so we can interact with the db
cursor = cnx.cursor()

In [ ]:
cursor.execute("DROP DATABASE sales;")

In [ ]:
# creating your database

db_name = 'sales'

def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

In [ ]:
try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exist.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)
        
cursor.close()
cnx.close()

In [ ]:
# creating tables

TABLES = {}
TABLES['customers'] = (
        "CREATE TABLE customers ("
        " customerNumber int(11) NOT NULL,"
        " customerName varchar(64) NOT NULL,"
        " contactLastName varchar(16) NOT NULL,"
        " contactFirstName varchar(16) NOT NULL,"
        " phone varchar(16) NOT NULL,"
        " addressLine1 varchar(64) NOT NULL,"
        " addressLine2 varchar(64) NOT NULL,"
        " city varchar(64) NOT NULL,"
        " state varchar(64) NOT NULL,"
        " postalCode int(10) NOT NULL,"
        " country varchar(64) NOT NULL,"
        " salesRepEmployeeNumber int(11) NOT NULL,"
        " creditLimit int(11) NOT NULL) ENGINE=InnoDB"
)



In [ ]:
customers.head()

In [ ]:
print(TABLES['customers'])

In [ ]:
# connecting to your database
cnx = mysql.connector.connect(
    host = 'test-db1.cpizore09tpz.us-east-2.rds.amazonaws.com',
    user = 'admin',
    passwd = 'testing123',
    database = 'sales'
)

# cursor so we can interact with the db
cursor = cnx.cursor()

In [ ]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()
cnx.close()

### Inserting Data

In [ ]:
customers.columns

In [ ]:
insert = ("INSERT INTO customers (customerNumber, "
"customerName, contactLastName, contactFirstName, phone, "
"addressLine1, addressLine2, city, state, postalCode, "
"country, salesRepEmployeeNumber, creditLimit) VALUES "
"(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")

In [ ]:
insert

In [ ]:
customers = customers.dropna()
customers['salesRepEmployeeNumber'] = customers['salesRepEmployeeNumber'].astype(int)
customers = customers.applymap(str)

In [ ]:
customers.head()

In [ ]:
first = tuple(customers.iloc[0].values)
first

In [ ]:
# connecting to your database
cnx = mysql.connector.connect(
    host = 'test-db1.cpizore09tpz.us-east-2.rds.amazonaws.com',
    user = 'admin',
    passwd = 'testing123',
    database = 'sales'
)

# cursor so we can interact with the db
cursor = cnx.cursor()

# this is the insert!
cursor.execute(insert, first)

# commiting changes and closing the connection
cnx.commit()
cursor.close()
cnx.close()

In [ ]:
data = [tuple(x) for x in customers.to_numpy()]
data[0]

In [ ]:
# connecting to your database
cnx = mysql.connector.connect(
    host = 'test-db1.cpizore09tpz.us-east-2.rds.amazonaws.com',
    user = 'admin',
    passwd = 'testing123',
    database = 'sales'
)

# cursor so we can interact with the db
cursor = cnx.cursor()

cursor.executemany(insert, data)

cnx.commit()
cursor.close()
cnx.close()

In [ ]:
# connecting to your database
cnx = mysql.connector.connect(
    host = 'test-db1.cpizore09tpz.us-east-2.rds.amazonaws.com',
    user = 'admin',
    passwd = 'testing123',
    database = 'sales'
)
cursor = cnx.cursor()

cursor.execute("SELECT * FROM customers;")
pd.DataFrame(cursor.fetchall(), columns=[x[0] for x in cursor.description])

You can either use the above method to read in your data, or use `pd.read_sql` (e